In [1]:
import pandas as pd

In [2]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# raise StopExecution

In [ ]:
parent_dir="/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/"

In [3]:
# note1!!!!!!!!! should be comfort file with list colunms writed as [....]
# note2!!! this code is assuming all the sites of genes have at least one common accession number - 
# here I only have one gene that are not compatible so I fixed it manualy
fp = parent_dir+"/nodups_cov5_HE_PLUS_STAR100len/comfort_ver_final_table_amino_change_by_site_byFreq.tab.csv"
# read table
sites = pd.read_csv(filepath_or_buffer=fp, sep="\t") 
# find genes with above 40 sites
clusters_ZNF = sites["gene_name"].value_counts().rename_axis('gene_name').reset_index(name='counts')
clusters_ZNF_above_40 = clusters_ZNF[clusters_ZNF["counts"] > 40]
# keep only sites of those genes
sites_above_40 = sites[sites["gene_name"].isin(clusters_ZNF_above_40['gene_name'])]
# split gene accesion to list
sites_above_40['genes']=sites_above_40['genes'].apply(eval)
sites_above_40['aminos_change']=sites_above_40['aminos_change'].apply(eval)
sites_above_40['mut_type']=sites_above_40['mut_type'].apply(eval)
sites_above_40['selected_acc']= ""
sites_above_40['selected_aminos_change']= ""
sites_above_40['selected_mut_type']= ""
clusters_ZNF_above_40['accession_n'] = ""

# choose acc number for every gene symbol 
dict_list_sites_above_40=[]
# manual fix gene with problem of accession number
# printing to ensure that exact 2 row was removed
print(sites_above_40.shape[0])
# sites_above_40 = sites_above_40[sites_above_40['Position'] != 35254398 ]
# sites_above_40 = sites_above_40[sites_above_40['Position'] != 35254274 ]
print(sites_above_40.shape[0])
#for symbol
for symb in clusters_ZNF_above_40["gene_name"]:
    # select symbol's lines
    symb_df = sites_above_40[sites_above_40['gene_name'] == symb]
    dict_list_symb_df=symb_df.to_dict('records')
    my_flag=False
    # find acc number that is in all the sites
    symb_acc_list = [acc for acc_list in symb_df['genes'] for acc in acc_list]
    symb_acc_set=set(symb_acc_list)
    for acc_n in symb_acc_set:       
        # filter sites that contains this acc n
        temp_df = symb_df[symb_df['genes'].apply(lambda x: acc_n in x)]
        # if its all the sites
        if temp_df.shape[0] == symb_df.shape[0]:
            for site in dict_list_symb_df:
                my_flag=True
                site['selected_acc']=acc_n
                clusters_ZNF_above_40.loc[clusters_ZNF_above_40['gene_name']==symb,'accession_n'] = acc_n
                # find acc index and save copmatible amino change
                for idx,gene in enumerate(site['genes']):
                    if gene==acc_n:
                            site['selected_aminos_change']=site['aminos_change'][idx]
                            site['selected_mut_type']=site['mut_type'][idx]
                            break
        # if we found acc number stop iterate over site's acc list
        if my_flag:
            break
    assert my_flag, "no acc for gene"+symb
    dict_list_sites_above_40.extend(dict_list_symb_df)
        
for site in dict_list_sites_above_40:
    assert len(site['selected_acc']) > 3, len(site['selected_acc'])
    assert site['selected_acc'] in site['genes'], str(site)


FileNotFoundError: [Errno 2] No such file or directory: '/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/nodups_cov5_HE_PLUS_STAR100len/comfort_ver_final_table_amino_change_by_site_byFreq.tab.csv'

write and make data easy for the next stage

In [13]:

genes_out_path = parent_dir + "/domain_editing/above_40_geneID.csv"
clusters_ZNF_above_40.to_csv(genes_out_path, index=False)
final_site_table = pd.DataFrame.from_dict(dict_list_sites_above_40)
final_site_table.rename(columns={'selected_aminos_change':'amino_change'}, inplace=True)
final_site_table["mut_type_short"]="nonsyn"
final_site_table.mut_type_short[final_site_table['selected_mut_type'] == 'synonymous_variant'] = 'syn'
sites_out_path = parent_dir+"/above_40_sites.csv"
final_site_table.to_csv(sites_out_path, index=False)


/tmp/ipykernel_8646/735495276.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_site_table.mut_type_short[final_site_table['selected_mut_type'] == 'synonymous_variant'] = 'syn'
